<a href="https://colab.research.google.com/github/EYBishal/Python-Training-Bootcamp/blob/main/15th-Jan-2026/Project1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName('1STpROJECT').getOrCreate()

In [3]:
from google.colab import files
upload=files.upload()

Saving orders.csv to orders.csv


In [78]:
path='/content/orders.csv'
df=spark.read.csv(path,header=True,inferSchema=False)
df.printSchema()
df.show(10)

root
 |-- order_id: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- city: string (nullable = true)
 |-- category: string (nullable = true)
 |-- product: string (nullable = true)
 |-- amount: string (nullable = true)
 |-- order_date: string (nullable = true)
 |-- status: string (nullable = true)

+-----------+-----------+-----------+-----------+-----------+-------+----------+---------+
|   order_id|customer_id|       city|   category|    product| amount|order_date|   status|
+-----------+-----------+-----------+-----------+-----------+-------+----------+---------+
|ORD00000000|    C000000| hyderabad |   grocery |       Oil |invalid|01/01/2024|Cancelled|
|ORD00000001|    C000001|       Pune|    Grocery|      Sugar|  35430|2024-01-02|Completed|
|ORD00000002|    C000002|       Pune|Electronics|     Mobile|  65358|2024-01-03|Completed|
|ORD00000003|    C000003|  Bangalore|Electronics|     Laptop|   5558|2024-01-04|Completed|
|ORD00000004|    C000004|       Pune|    

In [79]:
df.count()

300000

**Explain why all columns must be treated as StringType initially ?**

**Schema uncertainty:** Raw data often has inconsistent types; starting with strings avoids errors.

**Preserve data:** Prevents loss of values that don't match inferred types.

**Flexibility:** Easier to clean and validate before converting to correct types.

**Avoid job failures:** Loading as strings reduces parsing issues in big data pipelines.

**Custom transformations:** Gives full control for casting after data quality checks.


In [80]:
from pyspark.sql import functions as f
fix_col=['city','category','product']
for i in fix_col:
  df=df.withColumn(i,f.initcap(f.trim(i)))
df.show(10)

+-----------+-----------+---------+-----------+-----------+-------+----------+---------+
|   order_id|customer_id|     city|   category|    product| amount|order_date|   status|
+-----------+-----------+---------+-----------+-----------+-------+----------+---------+
|ORD00000000|    C000000|Hyderabad|    Grocery|        Oil|invalid|01/01/2024|Cancelled|
|ORD00000001|    C000001|     Pune|    Grocery|      Sugar|  35430|2024-01-02|Completed|
|ORD00000002|    C000002|     Pune|Electronics|     Mobile|  65358|2024-01-03|Completed|
|ORD00000003|    C000003|Bangalore|Electronics|     Laptop|   5558|2024-01-04|Completed|
|ORD00000004|    C000004|     Pune|       Home|Airpurifier|  33659|2024-01-05|Completed|
|ORD00000005|    C000005|    Delhi|    Fashion|      Jeans|   8521|2024-01-06|Completed|
|ORD00000006|    C000006|    Delhi|    Grocery|      Sugar|  42383|2024-01-07|Completed|
|ORD00000007|    C000007|     Pune|    Grocery|       Rice|  45362|2024-01-08|Completed|
|ORD00000008|    C000

In [81]:
re_amount=f.regexp_extract(f.col("amount"),r"(\d+)",0)
df=df.withColumn("amount",f.when((re_amount=="")|re_amount.isNull(),f.lit(0)).otherwise(re_amount.cast("int")))
df.select('amount').show(10)

+------+
|amount|
+------+
|     0|
| 35430|
| 65358|
|  5558|
| 33659|
|  8521|
| 42383|
| 45362|
| 10563|
| 63715|
+------+
only showing top 10 rows


In [142]:

df=df.withColumn("order_date",f.trim(f.col("order_date")))
df=df.withColumn("order_date_clean",f.coalesce(
f.expr("try_to_timestamp(order_date, 'yyyy-MM-dd')"),
        f.expr("try_to_timestamp(order_date, 'dd/MM/yyyy')"),
        f.expr("try_to_timestamp(order_date, 'yyyy/MM/dd')")
    ))

df.show(10)


DateTimeException: [CANNOT_PARSE_TIMESTAMP] Text '12/01/2024' could not be parsed at index 0. Use `try_to_timestamp` to tolerate invalid input string and return NULL instead. SQLSTATE: 22007

In [83]:
invalid_amount=df.filter(df.amount == 0).count()
print(invalid_amount)

25164


In [84]:
invalid_date=df.filter(df.order_date_clean.isNull()).count()
print(invalid_date)

2595


In [85]:
dup_df=df.groupBy('order_id').count().filter(f.col('count')>1)
dup_df.show()
df.count()

+--------+-----+
|order_id|count|
+--------+-----+
+--------+-----+



300000

In [86]:
df=df.drop_duplicates(['order_id'])
df.count()



300000

In [87]:
df=df.filter(f.col("status")=="Completed")
df.count()

285000

In [65]:

num_partitions = df.rdd.getNumPartitions()
print(f"Number of partitions: {num_partitions}")


Number of partitions: 2


In [88]:
city_df=df.groupBy('city').agg(f.sum('amount').alias('total_amount'))
city_df.show()

+---------+------------+
|     city|total_amount|
+---------+------------+
|Bangalore|  1608818821|
|  Chennai|  1609809323|
|   Mumbai|  1605761872|
|  Kolkata|  1604076741|
|     Pune|  1626056695|
|    Delhi|  1619020556|
|Hyderabad|  1622219584|
+---------+------------+



In [90]:
city_df.explain(True)

**4. Identify where shuffle happens.**

**Shuffle occurs at:**

Exchange hashpartitioning(city, 200) before final aggregation.

Exchange hashpartitioning(order_id, 200) during dedup/SortAggregate.

In [92]:

repart_dp=df.repartition("city")


In [93]:

print("Before repartition:")
df.explain(True)

print("After repartition:")
repart_dp.explain(True)




Before repartition:
== Parsed Logical Plan ==
'Filter '`=`('status, Completed)
+- Deduplicate [order_id#1419]
   +- Project [order_id#1419, customer_id#1420, city#1473, category#1474, product#1475, amount#1509, order_date#1515, status#1426, coalesce(try_to_timestamp(order_date#1515, Some(yyyy-MM-dd), TimestampType, Some(Etc/UTC), false), try_to_timestamp(order_date#1515, Some(dd/MM/yyyy), TimestampType, Some(Etc/UTC), false), try_to_timestamp(order_date#1515, Some(yyyy/MM/dd), TimestampType, Some(Etc/UTC), false)) AS order_date_clean#1516]
      +- Project [order_id#1419, customer_id#1420, city#1473, category#1474, product#1475, amount#1509, trim(order_date#1425, None) AS order_date#1515, status#1426]
         +- Project [order_id#1419, customer_id#1420, city#1473, category#1474, product#1475, CASE WHEN ((regexp_extract(amount#1424, (\d+), 0) = ) OR isnull(regexp_extract(amount#1424, (\d+), 0))) THEN 0 ELSE cast(regexp_extract(amount#1424, (\d+), 0) as int) END AS amount#1509, order_da

In [94]:
df_rev=df.groupBy('city').agg(f.sum('amount').alias('total_revenue'))
df_rev.show()

+---------+-------------+
|     city|total_revenue|
+---------+-------------+
|Bangalore|   1608818821|
|  Chennai|   1609809323|
|   Mumbai|   1605761872|
|  Kolkata|   1604076741|
|     Pune|   1626056695|
|    Delhi|   1619020556|
|Hyderabad|   1622219584|
+---------+-------------+



In [95]:
df_rev=df.groupBy('category').agg(f.sum('amount').alias('total_revenue'))
df_rev.show()

+-----------+-------------+
|   category|total_revenue|
+-----------+-------------+
|       Home|   2833870208|
|    Fashion|   2799225164|
|    Grocery|   2831327086|
|Electronics|   2831341134|
+-----------+-------------+



In [96]:
df_rev=df.groupBy('city').agg(f.avg('amount').alias('total_revenue'))
df_rev.show()

+---------+------------------+
|     city|     total_revenue|
+---------+------------------+
|Bangalore| 39910.16896132569|
|  Chennai|39518.100034367635|
|   Mumbai| 39539.09859154929|
|  Kolkata| 39545.31817173286|
|     Pune| 39773.41914732285|
|    Delhi| 39629.42566211387|
|Hyderabad| 39526.80451256061|
+---------+------------------+



In [97]:
df_rev=df.groupBy('product').agg(f.sum('amount').alias('total_revenue')).orderBy(f.desc('total_revenue')).limit(10)
df_rev.show()

+-----------+-------------+
|    product|total_revenue|
+-----------+-------------+
|        Oil|    951812641|
|     Laptop|    950556247|
|     Tablet|    948911819|
|     Vacuum|    948228359|
|      Mixer|    945319858|
|       Rice|    942865877|
|Airpurifier|    940321991|
|      Jeans|    939537887|
|      Sugar|    936648568|
|      Shoes|    935350562|
+-----------+-------------+



In [98]:
df_rev=df.groupBy('city').agg(f.sum('amount').alias('total_revenue')).orderBy(f.desc('total_revenue'))
df_rev.show()

+---------+-------------+
|     city|total_revenue|
+---------+-------------+
|     Pune|   1626056695|
|Hyderabad|   1622219584|
|    Delhi|   1619020556|
|  Chennai|   1609809323|
|Bangalore|   1608818821|
|   Mumbai|   1605761872|
|  Kolkata|   1604076741|
+---------+-------------+



Rank cities by revenue.
2. Rank products inside each category by revenue.
3. Find the top product for every category.
4. Identify the top 3 performing cities.

In [103]:
from pyspark.sql.window import Window
s=df.groupBy('city').agg(f.sum('amount').alias('total_amount'))
w = Window.orderBy(f.desc("total_amount"))
df_rev=s.withColumn("rank",f.rank().over(w))
df_rev.show()


+---------+------------+----+
|     city|total_amount|rank|
+---------+------------+----+
|     Pune|  1626056695|   1|
|Hyderabad|  1622219584|   2|
|    Delhi|  1619020556|   3|
|  Chennai|  1609809323|   4|
|Bangalore|  1608818821|   5|
|   Mumbai|  1605761872|   6|
|  Kolkata|  1604076741|   7|
+---------+------------+----+



In [107]:

product_rev = df.groupBy("category", "product").agg(f.sum("amount").alias("total_revenue"))
w = Window.partitionBy("category").orderBy(f.desc("total_revenue"))

ranked_products = product_rev.withColumn("rank", f.rank().over(w))
ranked_products.show()


+-----------+-----------+-------------+----+
|   category|    product|total_revenue|rank|
+-----------+-----------+-------------+----+
|Electronics|     Laptop|    950556247|   1|
|Electronics|     Tablet|    948911819|   2|
|Electronics|     Mobile|    931873068|   3|
|    Fashion|      Jeans|    939537887|   1|
|    Fashion|      Shoes|    935350562|   2|
|    Fashion|     Tshirt|    924336715|   3|
|    Grocery|        Oil|    951812641|   1|
|    Grocery|       Rice|    942865877|   2|
|    Grocery|      Sugar|    936648568|   3|
|       Home|     Vacuum|    948228359|   1|
|       Home|      Mixer|    945319858|   2|
|       Home|Airpurifier|    940321991|   3|
+-----------+-----------+-------------+----+



In [108]:
product_rev = df.groupBy("category", "product").agg(f.sum("amount").alias("total_revenue"))
w = Window.partitionBy("category").orderBy(f.desc("total_revenue"))

ranked_products = product_rev.withColumn("rank", f.rank().over(w)).filter(f.col("rank")==1)
ranked_products.show()

+-----------+-------+-------------+----+
|   category|product|total_revenue|rank|
+-----------+-------+-------------+----+
|Electronics| Laptop|    950556247|   1|
|    Fashion|  Jeans|    939537887|   1|
|    Grocery|    Oil|    951812641|   1|
|       Home| Vacuum|    948228359|   1|
+-----------+-------+-------------+----+



In [113]:
product_rev=df.groupBy("city").agg(f.sum("amount").alias("total_revenue"))
w=Window.orderBy(f.desc("total_revenue"))

ranked_products = product_rev.withColumn("rank", f.rank().over(w)).filter(f.col("rank")<=3)
ranked_products.show()

+---------+-------------+----+
|     city|total_revenue|rank|
+---------+-------------+----+
|     Pune|   1626056695|   1|
|Hyderabad|   1622219584|   2|
|    Delhi|   1619020556|   3|
+---------+-------------+----+



In [114]:

from pyspark.sql import Row

city_region_data = [
    Row(city="Delhi", region="North"),
    Row(city="Mumbai", region="West"),
    Row(city="Bangalore", region="South"),
    Row(city="Hyderabad", region="South"),
    Row(city="Pune", region="West"),
    Row(city="Chennai", region="South"),
    Row(city="Kolkata", region="East")
]

city_region_df = spark.createDataFrame(city_region_data)
city_region_df.show()

+---------+------+
|     city|region|
+---------+------+
|    Delhi| North|
|   Mumbai|  West|
|Bangalore| South|
|Hyderabad| South|
|     Pune|  West|
|  Chennai| South|
|  Kolkata|  East|
+---------+------+



In [115]:

from pyspark.sql.functions import broadcast

joined_df = df.join(broadcast(city_region_df), on="city", how="inner")


In [116]:
joined_df.explain(True)

== Parsed Logical Plan ==
'Join UsingJoin(Inner, [city])
:- Filter (status#1426 = Completed)
:  +- Deduplicate [order_id#1419]
:     +- Project [order_id#1419, customer_id#1420, city#1473, category#1474, product#1475, amount#1509, order_date#1515, status#1426, coalesce(try_to_timestamp(order_date#1515, Some(yyyy-MM-dd), TimestampType, Some(Etc/UTC), false), try_to_timestamp(order_date#1515, Some(dd/MM/yyyy), TimestampType, Some(Etc/UTC), false), try_to_timestamp(order_date#1515, Some(yyyy/MM/dd), TimestampType, Some(Etc/UTC), false)) AS order_date_clean#1516]
:        +- Project [order_id#1419, customer_id#1420, city#1473, category#1474, product#1475, amount#1509, trim(order_date#1425, None) AS order_date#1515, status#1426]
:           +- Project [order_id#1419, customer_id#1420, city#1473, category#1474, product#1475, CASE WHEN ((regexp_extract(amount#1424, (\d+), 0) = ) OR isnull(regexp_extract(amount#1424, (\d+), 0))) THEN 0 ELSE cast(regexp_extract(amount#1424, (\d+), 0) as int) EN

 **Explain why broadcast join is efficient in this case.**

The city-region dataset is very small (7 rows).

Spark broadcasts it to all executors → avoids shuffle → faster join.

In [117]:
from pyspark.sql.types import StringType
def amount_classify(amount):
  if amount >= 80000:
    return "High"
  elif amount >= 40000:
    return "Medium"
  else:
    return "Low"
classify_udf=f.udf(amount_classify,StringType())
df = df.withColumn("order_value_category", classify_udf(f.col("amount")))
df.groupBy("order_value_category").count().show()

+--------------------+------+
|order_value_category| count|
+--------------------+------+
|                High| 27936|
|                 Low|145699|
|              Medium|111365|
+--------------------+------+



In [118]:

rdd = df.rdd
print(rdd.take(5))


[Row(order_id='ORD00000001', customer_id='C000001', city='Pune', category='Grocery', product='Sugar', amount=35430, order_date='2024-01-02', status='Completed', order_date_clean=datetime.datetime(2024, 1, 2, 0, 0), order_value_category='Low'), Row(order_id='ORD00000007', customer_id='C000007', city='Pune', category='Grocery', product='Rice', amount=45362, order_date='2024-01-08', status='Completed', order_date_clean=datetime.datetime(2024, 1, 8, 0, 0), order_value_category='Medium'), Row(order_id='ORD00000008', customer_id='C000008', city='Bangalore', category='Fashion', product='Jeans', amount=10563, order_date='2024-01-09', status='Completed', order_date_clean=datetime.datetime(2024, 1, 9, 0, 0), order_value_category='Low'), Row(order_id='ORD00000010', customer_id='C000010', city='Bangalore', category='Grocery', product='Sugar', amount=66576, order_date='2024-01-11', status='Completed', order_date_clean=datetime.datetime(2024, 1, 11, 0, 0), order_value_category='Medium'), Row(order_i

In [122]:
saly=df.select('amount').rdd.map(lambda x:x[0])
tot=saly.reduce(lambda x,y:x+y)
print(tot)

11295763592


In [123]:

orders_per_city = (
    df.select("city").rdd
      .map(lambda r: (r[0], 1))
      .reduceByKey(lambda a, b: a + b)
)
print(orders_per_city.take(10))


[('Pune', 40883), ('Mumbai', 40612), ('Hyderabad', 41041), ('Delhi', 40854), ('Bangalore', 40311), ('Kolkata', 40563), ('Chennai', 40736)]


**Why DataFrames are preferred over RDDs for analytics (short)**

**Columnar memory + off-heap execution:** Better CPU cache usage, compression, and vectorized processing → lower memory and CPU cost for scans/aggregations.

**Declarative, concise, and safer:** High-level APIs (select, groupBy, agg) reduce boilerplate and bugs, and integrate seamlessly with Spark SQL.

**Identify datasets reused in multiple queries.**

orders_df = df.filter(F.col("status") == "Completed")

In [124]:
df.cache()

DataFrame[order_id: string, customer_id: string, city: string, category: string, product: string, amount: int, order_date: string, status: string, order_date_clean: timestamp, order_value_category: string]

In [125]:
df.unpersist()

DataFrame[order_id: string, customer_id: string, city: string, category: string, product: string, amount: int, order_date: string, status: string, order_date_clean: timestamp, order_value_category: string]

In [131]:

import time

orders_df = df.filter(f.col("status") == "Completed")

# ---------------- BEFORE CACHING ----------------
start = time.time()

orders_df.agg(f.sum("amount").alias("total_revenue"))

orders_df.groupBy("city").count()

orders_df.groupBy("category").agg(f.sum("amount").alias("category_revenue"))

end = time.time()
print(f"Time before caching: {end - start:.2f} seconds")

# ---------------- APPLY CACHE ----------------
orders_df.cache()

# ---------------- AFTER CACHING ----------------
start = time.time()

orders_df.agg(f.sum("amount").alias("total_revenue"))
orders_df.groupBy("city").count()
orders_df.groupBy("category").agg(f.sum("amount").alias("category_revenue"))

end = time.time()
print(f"Time after caching: {end - start:.2f} seconds")

# ---------------- UNPERSIST ----------------
orders_df.unpersist()


Time before caching: 0.05 seconds
Time after caching: 0.03 seconds


DataFrame[order_id: string, customer_id: string, city: string, category: string, product: string, amount: int, order_date: string, status: string, order_date_clean: timestamp, order_value_category: string]

**Explain why unnecessary caching is dangerous.**

**Consumes memory:** Cached data stays in RAM, reducing space for other tasks.

**May cause spills:** If memory is full, Spark spills to disk → slows down jobs.

**Wasted resources:** If cached data is never reused, caching adds overhead without benefit.

In [132]:
df.write.mode("overwrite").partitionBy("city").parquet("/path/cleaned_orders_parquet")

In [134]:

agg_df = df.groupBy("city").agg(f.sum("amount").alias("total_revenue"))
agg_df.write.mode("overwrite").orc("/path/aggregated_orders_orc")


In [135]:

parquet_df = spark.read.parquet("/path/cleaned_orders_parquet")
orc_df = spark.read.orc("/path/aggregated_orders_orc")

parquet_df.printSchema()
orc_df.printSchema()



root
 |-- order_id: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- category: string (nullable = true)
 |-- product: string (nullable = true)
 |-- amount: integer (nullable = true)
 |-- order_date: string (nullable = true)
 |-- status: string (nullable = true)
 |-- order_date_clean: timestamp (nullable = true)
 |-- order_value_category: string (nullable = true)
 |-- city: string (nullable = true)

root
 |-- city: string (nullable = true)
 |-- total_revenue: long (nullable = true)



**Explain why this breaks:**

**df = df.filter(df.amount > 50000).show()**
**And why after this line df is no longer a DataFrame.**

.show() returns None, not a DataFrame.
 df is None, so df is no longer a DataFrame.



In [136]:
df.printSchema()

critical_fields = ["order_id", "amount", "order_date_clean"]
for col in critical_fields:
    print(f"{col} null count:", df.filter(f.col(col).isNull()).count())


root
 |-- order_id: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- city: string (nullable = true)
 |-- category: string (nullable = true)
 |-- product: string (nullable = true)
 |-- amount: integer (nullable = true)
 |-- order_date: string (nullable = true)
 |-- status: string (nullable = true)
 |-- order_date_clean: timestamp (nullable = true)
 |-- order_value_category: string (nullable = true)

order_id null count: 0
amount null count: 0
order_date_clean null count: 2465


**Document:**

**Cleaning strategy:**

Normalized dates, handled invalid placeholders, cast amount to integer, removed nulls.


**Performance strategy:**

Used cache() for reused datasets, partitioned writes, broadcast joins for small lookup tables.


**Debugging learnings:**

Avoid chaining .show() when assigning DataFrames.
Validate schema after transformations.
Use Spark UI for performance tuning.